# Exercise 15

In [31]:
import pinocchio as pin
import numpy as np

### Define parameters

In [32]:
theta_1 = np.pi / 4
theta_2 = np.pi / 8

theta_1_dot = 0
theta_2_dot = 0

L_1 = 1
L_2 = 1

m_1 = 1
m_2 = 1

tau_1 = 1
tau_2 = 1

In [33]:
M_inertia = np.array([
    [m_1 * L_1**2 + m_2*(L_1**2 + 2*L_1*L_2*np.cos(theta_2) + L_2**2), m_2 *(L_1*L_2 * np.cos(theta_2) + L_2**2) ],
    [m_2 *(L_1*L_2 * np.cos(theta_2) + L_2**2), m_2 * L_2**2]
])

c = np.array([
    [-m_2 * L_1 * L_2 * np.sin(theta_2)* (2*theta_1_dot *theta_2_dot )],
    [m_2*L_1*L_2*np.sin(theta_2)*theta_1_dot]
])

g = np.array([
    [(m_1 + m_2) * 9.81 * L_1 * np.cos(theta_1) + m_2 * 9.81 * L_2 * np.cos(theta_1 + theta_2)],
    [m_2 * 9.81 * L_2 * np.cos(theta_1 + theta_2)]
])

print('M(theta) = \n', M_inertia)
print('c(theta, theta_dot) = \n', c)
print('g(theta) = \n', g)

M(theta) = 
 [[4.84775907 1.92387953]
 [1.92387953 1.        ]]
c(theta, theta_dot) = 
 [[-0.]
 [ 0.]]
g(theta) = 
 [[17.62755952]
 [ 3.75412447]]


### b) Forward kinematics

In [34]:
S_1 = np.array([0, 0, 0, 0, 0, 1]) # different notation then lectures -> [v, w] !!
S_2 = np.array([0, -1, 0, 0, 0, 1])

# When theta_1 = 0, theta_2 = 0, this is the transformation matrix
M = np.array([[1, 0, 0, L_1 + L_2], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

# Calculate the transformation matrix for the given thetas via PoE
T_sb = pin.exp6(S_1 * theta_1).homogeneous @ pin.exp6(S_2 * theta_2).homogeneous  @ M
print("T_sb:")
print(T_sb)

print('\nPosition of end effector\n', pin.SE3(T_sb).translation)

T_sb:
[[ 0.38268343 -0.92387953  0.          1.08979021]
 [ 0.92387953  0.38268343  0.          1.63098631]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]

Position of end effector
 [1.08979021 1.63098631 0.        ]


### c) Space and body Jacobian

In [35]:
J_s1 = S_1
J_s2 = pin.exp6(S_1 * theta_1).action @ S_2

J_s = np.column_stack((J_s1, J_s2))
print("J_s:")
print(J_s)

J_b = pin.SE3(T_sb).inverse().action @ J_s
print("J_b:")
print(J_b)

J_s:
[[ 0.          0.70710678]
 [ 0.         -0.70710678]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 1.          1.        ]]
J_b:
[[3.82683432e-01 1.66533454e-16]
 [1.92387953e+00 1.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 1.00000000e+00]]


### e) Forward dynamics

In [36]:
ddq = np.linalg.solve(M_inertia, np.array([[tau_1], [tau_2]]) - c - g)
print("ddq:")
print(ddq)

ddq:
[[-9.88179975]
 [16.25726781]]


### f) Solve above questions with Pinocchio framework (i.e. via URDF)

In [37]:
urdf_model_path = "2R.urdf"

# Load the urdf model
model, collision_model, visual_model = pin.buildModelsFromUrdf(
    urdf_model_path
)
print("model name: " + model.name)

# Set gravity to y axis since the robot is planar!
model.gravity = pin.Motion(np.array([0, -9.81, 0]), np.zeros(3))


# Create data required by the algorithms
data, collision_data, visual_data = pin.createDatas(
    model, collision_model, visual_model
)

q = np.array([[theta_1], [theta_2]])
ee_frame_name = "point_mass_2"

model name: 2R robot arm


Visualize the robot to see if it is implemented correctly.

In [38]:
from pinocchio.visualize import MeshcatVisualizer as PMV

viz = PMV(model, collision_model, visual_model, collision_data=collision_data, visual_data=visual_data)
viz.initViewer(open=False)

viz.loadViewerModel()
# Display a robot configuration.
viz.display(q)
viz.displayVisuals(True)
viz.displayFrames(True)
viz.updateFrames()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/


In [39]:
# Perform the forward kinematics over the kinematic tree
pin.forwardKinematics(model, data, q)
pin.updateFramePlacements(model, data)

# Get the transformation matrix from the world to the frame
T_pin = data.oMf[model.getFrameId(ee_frame_name)]
print("T_sb:")
print(T_pin)

T_sb:
  R =
0.382683 -0.92388        0
 0.92388 0.382683        0
       0        0        1
  p = 1.08979 1.63099       0



In [40]:
pin.computeJointJacobians(model, data, q)
print("J_s:") 
print(data.J)

pin.computeFrameJacobian(model, data, q, model.getFrameId(ee_frame_name), pin.LOCAL)
J_pin = pin.getFrameJacobian(model, data, model.getFrameId(ee_frame_name), pin.LOCAL) 
print("J_b:")
print(J_pin)

J_s:
[[ 0.          0.70710678]
 [ 0.         -0.70710678]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 1.          1.        ]]
J_b:
[[3.82683432e-01 5.55111512e-17]
 [1.92387953e+00 1.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 1.00000000e+00]]


In [41]:
ddq_pin = pin.aba(model, data, q, np.zeros(2), np.array([[tau_1],[tau_2]]))
print("ddq:")
print(ddq_pin)

ddq:
[-9.88179975 16.25726781]
